# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [ ]:
import logging
import requests 


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
requests.packages.urllib3.add_stderr_logger()

In [ ]:
#pip install --upgrade langchain

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
# set to True to use Ollama (and download the mxbai-embed-large model)
use_Ollama = True

import openai
openai.api_base = "https://api.openai.com/v1"
openai.api_base = os.environ['OPENAI_API_KEY']
embeddings_model_name = "text-embedding-ada-002"


if(use_Ollama):
    openai.api_base = "http://localhost:11434/v1"
    openai.api_key = "ollama"
    embeddings_model_name = "mxbai-embed-large"

print('API Key:', openai.api_key)
print('API Url:', openai.api_base)
print('Embeddings Model:', embeddings_model_name)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings


if(use_Ollama):
    embeddings_model = OllamaEmbeddings(model=embeddings_model_name)
else:
    embeddings_model = OpenAIEmbeddings(model=embeddings_model_name)

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
class LimitedCSVLoader(CSVLoader):
    def load(self):
        # Call the original load method to get all rows
        all_rows = super().load()

        # Restrict to the first 5 rows
        limited_rows = all_rows[:2]
        return limited_rows

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [ ]:
loader = LimitedCSVLoader(file_path=file, encoding='utf-8')

In [ ]:
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [ ]:
#pip install docarray
#pip install pydantic==1.10.8

In [ ]:
vs  = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings_model
)

In [ ]:
index = vs.from_loaders([loader])

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings_model
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0, api_key=openai.api_key, base_url=openai.api_base)

In [ ]:
response = index.query(question=query, llm=llm)

In [ ]:
display(Markdown(response))

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1/',
    api_key='ollama', # required, but unused
)


In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Say this is a test',
        }
    ],
    model='gemma:2b',
)

In [ ]:
chat_completion.choices[0].message.content

In [ ]:
query ="Please tell me more about Women's Campside Oxfords."
response = index.query(question=query, llm=llm)

In [ ]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0)


In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.invoke(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])